## Vamos trabalhar com os dados abertos do ONS - Operador Nacional do Sistema Elétrico.

### Optei por trabalhar com os dados de Carga de Energia. Definição conforme site: [Dados de carga por subsistema numa data de referência em base diária.](https://dados.ons.org.br/dataset/carga-energia)

### Nessa etapa, iremos validar as novas variáveis criadas.

### Load das bibliotecas e definição do contexto spark.

In [1]:
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from random import randint

In [2]:
sc = SparkContext.getOrCreate()

spark = (SparkSession
      .builder
      .getOrCreate()
     )

print('ApplicationID:', sc.applicationId)

ApplicationID: local-1632184203374


### Caminhos de Leitura e Load da base

In [3]:
read_path = '../data/refined/03_enriquecimento'

In [4]:
df = spark.read.parquet(read_path)

print(f'Número de registros: {df.count()}')

df.printSchema()

Número de registros: 31649
root
 |-- id_medicao: integer (nullable = true)
 |-- nom_subsistema: string (nullable = true)
 |-- id_subsistema: byte (nullable = true)
 |-- val_cargaenergiamwmed: double (nullable = true)
 |-- din_instante: date (nullable = true)
 |-- mes: integer (nullable = true)
 |-- ano: integer (nullable = true)



### Análise da completude das variáveis criadas

In [5]:
new_var = ['id_medicao', 'mes', 'ano']

for variavel in new_var:
    print(f'Quantidade de registros nulos da variável "{variavel}": \
    {df.filter(F.col(variavel).isNull()).count()}')

Quantidade de registros nulos da variável "id_medicao":     0
Quantidade de registros nulos da variável "mes":     0
Quantidade de registros nulos da variável "ano":     0


### Análise de 'id_medicao'
- Únicos, Valor mínimo, Máximo
- Verificação se os id's estão em ordem crescente de acordo com 'din_instante' e 'id_subsistema'

In [6]:
print(f'Quantidade de registros únicos: {df.select("id_medicao").dropDuplicates().count()}')

id_minimo = df.select(F.min("id_medicao")).collect()[0][0]
print(f'Id mínimo: {id_minimo}')

id_maximo = df.select(F.max("id_medicao")).collect()[0][0]
print(f'Id máximo: {id_maximo}')

Quantidade de registros únicos: 31649
Id mínimo: 1
Id máximo: 31649


In [7]:
df.select('id_medicao', 'din_instante', 'id_subsistema').show()

+----------+------------+-------------+
|id_medicao|din_instante|id_subsistema|
+----------+------------+-------------+
|         1|  2000-01-01|            1|
|         2|  2000-01-01|            2|
|         3|  2000-01-01|            3|
|         4|  2000-01-01|            4|
|         5|  2000-01-02|            1|
|         6|  2000-01-02|            2|
|         7|  2000-01-02|            3|
|         8|  2000-01-02|            4|
|         9|  2000-01-03|            1|
|        10|  2000-01-03|            2|
|        11|  2000-01-03|            3|
|        12|  2000-01-03|            4|
|        13|  2000-01-04|            1|
|        14|  2000-01-04|            2|
|        15|  2000-01-04|            3|
|        16|  2000-01-04|            4|
|        17|  2000-01-05|            1|
|        18|  2000-01-05|            2|
|        19|  2000-01-05|            3|
|        20|  2000-01-05|            4|
+----------+------------+-------------+
only showing top 20 rows



In [8]:
df.select('id_medicao', 'din_instante', 'id_subsistema').orderBy(F.col('id_medicao').desc()).show()

+----------+------------+-------------+
|id_medicao|din_instante|id_subsistema|
+----------+------------+-------------+
|     31649|  2021-08-29|            4|
|     31648|  2021-08-29|            3|
|     31647|  2021-08-29|            2|
|     31646|  2021-08-29|            1|
|     31645|  2021-08-28|            4|
|     31644|  2021-08-28|            3|
|     31643|  2021-08-28|            2|
|     31642|  2021-08-28|            1|
|     31641|  2021-08-27|            4|
|     31640|  2021-08-27|            3|
|     31639|  2021-08-27|            2|
|     31638|  2021-08-27|            1|
|     31637|  2021-08-26|            4|
|     31636|  2021-08-26|            3|
|     31635|  2021-08-26|            2|
|     31634|  2021-08-26|            1|
|     31633|  2021-08-25|            4|
|     31632|  2021-08-25|            3|
|     31631|  2021-08-25|            2|
|     31630|  2021-08-25|            1|
+----------+------------+-------------+
only showing top 20 rows



### Análise de 'mes'
- Valores únicos
- Verificar se a informação é compatível com 'din_instante'
- Distribuição da variável

In [9]:
print(f'Lista dos possíveis valores da variável "mes":')
mes_unicos = df.select('mes').dropDuplicates().collect()
mes_unicos = sorted([x[0] for x in mes_unicos])
print(mes_unicos)

Lista dos possíveis valores da variável "mes":
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [10]:
print(f'Quantidade de registros em que "mes" não é o mês obtido em "din_instante": \
{df.filter(F.col("mes") != F.month(F.col("din_instante"))).count()}')

Quantidade de registros em que "mes" não é o mês obtido em "din_instante": 0


In [11]:
mes_teste = randint(1, 12)
df.select('mes', 'din_instante').filter(F.col('mes') == mes_teste).show()

+---+------------+
|mes|din_instante|
+---+------------+
|  9|  2000-09-01|
|  9|  2000-09-01|
|  9|  2000-09-01|
|  9|  2000-09-01|
|  9|  2000-09-02|
|  9|  2000-09-02|
|  9|  2000-09-02|
|  9|  2000-09-02|
|  9|  2000-09-03|
|  9|  2000-09-03|
|  9|  2000-09-03|
|  9|  2000-09-03|
|  9|  2000-09-04|
|  9|  2000-09-04|
|  9|  2000-09-04|
|  9|  2000-09-04|
|  9|  2000-09-05|
|  9|  2000-09-05|
|  9|  2000-09-05|
|  9|  2000-09-05|
+---+------------+
only showing top 20 rows



In [12]:
df.groupBy('mes').count().orderBy(F.col('mes')).show()

+---+-----+
|mes|count|
+---+-----+
|  1| 2728|
|  2| 2489|
|  3| 2728|
|  4| 2640|
|  5| 2728|
|  6| 2640|
|  7| 2728|
|  8| 2720|
|  9| 2520|
| 10| 2604|
| 11| 2520|
| 12| 2604|
+---+-----+



### Análise de 'ano'
- Valores únicos
- Verificar se a informação é compatível com 'din_instante'
- Distribuição da variável

In [13]:
print(f'Lista dos possíveis valores da variável "ano":')
ano_unicos = df.select('ano').dropDuplicates().collect()
ano_unicos = sorted([x[0] for x in ano_unicos])
print(ano_unicos)

Lista dos possíveis valores da variável "ano":
[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


In [14]:
print(f'Quantidade de registros em que "ano" não é o ano obtido em "din_instante": \
{df.filter(F.col("ano") != F.year(F.col("din_instante"))).count()}')

Quantidade de registros em que "ano" não é o ano obtido em "din_instante": 0


In [15]:
ano_teste = randint(ano_unicos[0], ano_unicos[-1])
df.select('ano', 'din_instante').filter(F.col('ano') == ano_teste).show()

+----+------------+
| ano|din_instante|
+----+------------+
|2012|  2012-01-01|
|2012|  2012-01-01|
|2012|  2012-01-01|
|2012|  2012-01-01|
|2012|  2012-01-02|
|2012|  2012-01-02|
|2012|  2012-01-02|
|2012|  2012-01-02|
|2012|  2012-01-03|
|2012|  2012-01-03|
|2012|  2012-01-03|
|2012|  2012-01-03|
|2012|  2012-01-04|
|2012|  2012-01-04|
|2012|  2012-01-04|
|2012|  2012-01-04|
|2012|  2012-01-05|
|2012|  2012-01-05|
|2012|  2012-01-05|
|2012|  2012-01-05|
+----+------------+
only showing top 20 rows



In [16]:
df.groupBy('ano').count().orderBy(F.col('ano')).show(22)

+----+-----+
| ano|count|
+----+-----+
|2000| 1464|
|2001| 1460|
|2002| 1460|
|2003| 1460|
|2004| 1464|
|2005| 1460|
|2006| 1460|
|2007| 1460|
|2008| 1464|
|2009| 1460|
|2010| 1460|
|2011| 1460|
|2012| 1464|
|2013| 1461|
|2014| 1460|
|2015| 1460|
|2016| 1464|
|2017| 1460|
|2018| 1460|
|2019| 1460|
|2020| 1464|
|2021|  964|
+----+-----+



In [17]:
sc.stop()